In [ ]:
import os
from datetime import datetime
import pickle as pkl

import re
import multiprocessing

In [ ]:
THREAD_NUM = 4

In [ ]:
with open('phrase_list_total.txt', 'r') as fp:
    phrase_list_total = fp.read().split('\n')
fp.close()
phrase_list_total = [ _ for _ in phrase_list_total if len(_) ]

In [ ]:
MAX_PID = len(phrase_list_total) - 1
print('total size of phrase_list_total:', MAX_PID+1)

In [ ]:
PROJ_NAME_LIST = [
    'ACSMHK',
    'CBI',
    'CGST',
    'DSCCC',
    'FLWC',
    'FVC',
    'HKBC',
    'JNG',
    'KFC',
    'STBC',
    'WWBS',
    'YFCX',
    'YOS'
]

In [ ]:
sermonpathfilelist = []
for PROJ_NAME in PROJ_NAME_LIST:
    data_dir_name_curr = '../../data/' + PROJ_NAME + '/'
    sermonpathfilelist += [ 
        data_dir_name_curr + _ 
        for _ in os.listdir(data_dir_name_curr)
    ]
sermonpathfilelist = sorted(sermonpathfilelist)

In [ ]:
dict_sid2spfn = {} # dictionary mapping of sermon-id to sermon-pathfilename
for sid, sermonpathfilename in enumerate(sermonpathfilelist):
    dict_sid2spfn[sid] = sermonpathfilename

MAX_SID = sid
print('maximum sermon-id : %d' % MAX_SID)

In [ ]:
def get_sermontext(insid):
    input_path_file_name = dict_sid2spfn.get(insid)
    with open(input_path_file_name, 'r') as fp:
        text = fp.read().replace('\n', '').strip()
    fp.close()
    return text

In [ ]:
with open('../rep_common.txt', 'r') as fp:
    REP_LIST = [ _.strip() for _ in fp.readlines() ]
fp.close()
REP_LIST = [ _.split(", '", 1) for _ in REP_LIST ]
REP_LIST = [ [_[0], _[1][:-1]] for _ in REP_LIST ]

In [ ]:
def cleanse_special_char(inputText):
    txt2 = inputText
    for rep_ in REP_LIST:
        txt2 = txt2.replace(rep_[0], rep_[1])
    txt2 = txt2.strip()
    return txt2

In [ ]:
def symbol_removal(inputText):
    txt2 = inputText\
        .replace(' ', '').replace('.', '').replace(',', '') \
        .replace('?', '').replace('!', '').replace('-', '') \
        .replace('(', '').replace(')', '').replace('+', '') \
        .replace('[', '').replace(']', '').replace('*', '') \
        .replace('^', '').replace('$', '').replace('%', '') \
        .replace('#', '').replace('@', '') \
        .replace('\\', '')
    return txt2

In [ ]:
# sermonText-to-phrase matrix dimension:
# MAX_SID x MAX_PID

dict_sid2s = {}

In [ ]:
def scan_through(sid_init, q):
    for sid in range(sid_init, MAX_SID + 1, THREAD_NUM):
        if sid % 100 == 0:
            print('%s    progress %d / %d ...' % (str(datetime.now()), sid, MAX_SID))
        dict_sid2s[sid] = symbol_removal(
            cleanse_special_char(
                get_sermontext(
                    sid
                )
            )
        )
        for pid, p_curr in enumerate(phrase_list_total):
            s_curr = dict_sid2s[sid]
            if p_curr in s_curr:
                # pass
                n_ = re.findall(p_curr, s_curr)
                if n_: # if non-zero n)
                    q.put((sid, pid, n_))
    return

In [ ]:
# ===========================
# threading manipulaion

# to record the index of non-zero sparse element
# expected data structure: [ (sid_1,pid_1,n1), (sid2,pid2,n2), ... ]

sparse_tuple_Queue = [None] * THREAD_NUM
for i in range(THREAD_NUM):
    sparse_tuple_Queue[i] = multiprocessing.Queue()
    sparse_tuple_Queue[i].put([])
# END threading manipulaion
# ===========================

# ===========================
# threading manipulaion
p = [None] * THREAD_NUM
for i in range(THREAD_NUM):
    p[i] = multiprocessing.Process(
        target=scan_through,
        args=(
            i,
            sparse_tuple_Queue[i]
        )
    )
    p[i].start()
for i in range(THREAD_NUM):
    p[i].join()
# END threading manipulaion
# ===========================